In [1]:
#vosk
from vosk import Model, KaldiRecognizer, SetLogLevel
from pydub import AudioSegment
import subprocess
import json
import os


#ru_punct
from __future__ import division

import sys
sys.path.append("/home/jupyter/datasphere/project/ru_punct-main/")

import models
import data
import main
import sys
import codecs
import tensorflow as tf
import numpy as np


#saiga
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

2023-11-25 08:52:22.494683: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-25 08:52:23.764011: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/jupyter/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/home/jupyter/.local/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/home/jupyter/.local/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


In [2]:
MODEL_NAME_SAIGA = "IlyaGusev/saiga2_7b_lora"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        start_token_id=1,
        bot_token_id=9225
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += tokenizer.decode([self.start_token_id, self.bot_token_id])
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()



In [3]:
config_saiga = PeftConfig.from_pretrained(MODEL_NAME_SAIGA)
model_saiga = AutoModelForCausalLM.from_pretrained(
    config_saiga.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

adapter_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/554 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [5]:
model_saiga = PeftModel.from_pretrained(
    model_saiga,
    MODEL_NAME_SAIGA,
    torch_dtype=torch.float16
)

adapter_model.bin:   0%|          | 0.00/67.2M [00:00<?, ?B/s]

In [6]:
model_saiga.eval()

tokenizer_saiga = AutoTokenizer.from_pretrained(MODEL_NAME_SAIGA, use_fast=False)
generation_config_saiga = GenerationConfig.from_pretrained(MODEL_NAME_SAIGA)
print(generation_config_saiga)

tokenizer_config.json:   0%|          | 0.00/278 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 3584,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.2,
  "temperature": 0.5,
  "top_k": 30,
  "top_p": 0.9
}



In [21]:
#vosk ()

file_name='1.mp3'

SetLogLevel(0)

# Проверяем наличие модели
if not os.path.exists("model"):
    print ("Please download the model from https://alphacephei.com/vosk/models and unpack as 'model' in the current folder.")
    exit (1)

# Устанавливаем Frame Rate
FRAME_RATE = 16000
CHANNELS=1

model_vosk = Model("model")
rec = KaldiRecognizer(model_vosk, FRAME_RATE)
rec.SetWords(True)


def vosk_analysis(file_name):
    # Используя библиотеку pydub делаем предобработку аудио
    mp3 = AudioSegment.from_mp3(file_name)
    mp3 = mp3.set_channels(CHANNELS)
    mp3 = mp3.set_frame_rate(FRAME_RATE)

    # Преобразуем вывод в json
    rec.AcceptWaveform(mp3.raw_data)
    result = rec.Result()
    text = json.loads(result)["text"]
    return(text)

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from model/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from model/graph/HCLr.fst model/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo model/graph/phones/word_boundary.int


In [10]:
#ru_punct preparing

MAX_SUBSEQUENCE_LEN = 200
model_file_ru_punct = '/home/jupyter/datasphere/project/ru_punct-main/Model_ru_punctuator_h256_lr0.02.pcl'

def to_array(arr, dtype=np.int32):
    # minibatch of 1 sequence as column
    return np.array([arr], dtype=dtype).T

def convert_punctuation_to_readable(punct_token):
    if punct_token == data.SPACE:
        return " "
    else:
        return punct_token[0]

def restore(text, word_vocabulary, reverse_punctuation_vocabulary, model):
    i = 0
    while True:
        string_to_punct = ''
        subsequence = text[i:i+MAX_SUBSEQUENCE_LEN]

        if len(subsequence) == 0:
            break

        converted_subsequence = [word_vocabulary.get(w, word_vocabulary[data.UNK]) for w in subsequence]

        y = predict(to_array(converted_subsequence), model)

        string_to_punct += subsequence[0]

        last_eos_idx = 0
        punctuations = []
        for y_t in y:

            p_i = np.argmax(tf.reshape(y_t, [-1]))
            punctuation = reverse_punctuation_vocabulary[p_i]

            punctuations.append(punctuation)

            if punctuation in data.EOS_TOKENS:
                last_eos_idx = len(punctuations) # we intentionally want the index of next element

        if subsequence[-1] == data.END:
            step = len(subsequence) - 1
        elif last_eos_idx != 0:
            step = last_eos_idx
        else:
            step = len(subsequence) - 1

        for j in range(step):
            string_to_punct += (punctuations[j] + " " if punctuations[j] != data.SPACE else " ")
            if j < step - 1:
                string_to_punct += subsequence[1+j]

        if subsequence[-1] == data.END:
            break

        i += step
    return(string_to_punct)

def predict(x, model):
    return tf.nn.softmax(net(x))

In [11]:
#ru_punct model preparing

vocab_len_ru_punct = len(data.read_vocabulary(data.WORD_VOCAB_FILE))
x_len = vocab_len_ru_punct if vocab_len_ru_punct < data.MAX_WORD_VOCABULARY_SIZE else data.MAX_WORD_VOCABULARY_SIZE + data.MIN_WORD_COUNT_IN_VOCAB
x = np.ones((x_len, main.MINIBATCH_SIZE)).astype(int)

print("Loading model parameters...")
net, _ = models.load(model_file_ru_punct, x)

print("Building model...")

word_vocabulary = net.x_vocabulary
punctuation_vocabulary = net.y_vocabulary

reverse_word_vocabulary = {v:k for k,v in word_vocabulary.items()}
reverse_punctuation_vocabulary = {v:k for k,v in punctuation_vocabulary.items()}
for key, value in reverse_punctuation_vocabulary.items():
    if value == '.PERIOD':
        reverse_punctuation_vocabulary[key] = '.'
    if value == ',COMMA':
        reverse_punctuation_vocabulary[key] = ','
    if value == '?QUESTIONMARK':
        reverse_punctuation_vocabulary[key] = '?'

Vocabulary "/home/jupyter/datasphere/project/ru_punct-main/content/data_rpocessed/vocabulary" size: 100002
Loading model parameters...


2023-11-25 09:01:43.970953: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 23038 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:8c:00.0, compute capability: 7.0


Vocabulary "/home/jupyter/datasphere/project/ru_punct-main/content/data_rpocessed/vocabulary" size: 100002
Vocabulary "/home/jupyter/datasphere/project/ru_punct-main/content/data_rpocessed/punctuations" size: 4
[TensorShape([100002, 256]), TensorShape([256, 4]), TensorShape([1, 4]), TensorShape([256, 512]), TensorShape([512, 512]), TensorShape([1, 512]), TensorShape([512]), TensorShape([256, 256]), TensorShape([512, 256]), TensorShape([256, 256]), TensorShape([1, 256]), TensorShape([512, 512]), TensorShape([256, 512]), TensorShape([1, 512]), TensorShape([512, 256]), TensorShape([256, 256]), TensorShape([1, 256]), TensorShape([256, 512]), TensorShape([256, 512]), TensorShape([1, 512]), TensorShape([256, 256]), TensorShape([256, 256]), TensorShape([1, 256]), TensorShape([256, 512]), TensorShape([256, 512]), TensorShape([1, 512]), TensorShape([256, 256]), TensorShape([256, 256]), TensorShape([1, 256])]
Building model...


In [18]:
#ru_punct text analysing


def ru_punct_analysis(text):
    h=177
    output_text=''
    for i in range(0,len(text.split()),h):
        input_text = text.split()[i:i+h]
        text2 = [w for w in input_text if w not in punctuation_vocabulary and w not in data.PUNCTUATION_MAPPING and not w.startswith(data.PAUSE_PREFIX)] + [data.END]
        pauses = [float(s.replace(data.PAUSE_PREFIX,"").replace(">","")) for s in input_text if s.startswith(data.PAUSE_PREFIX)]


        text_with_punct = restore(text2, word_vocabulary, reverse_punctuation_vocabulary, net)
        import nltk.data
        punkt_tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')
        sentences = punkt_tokenizer.tokenize(text_with_punct)
        sentences = [sent.capitalize() for sent in sentences]
        uppercase_text = ' '.join(sentences)
        output_text+=uppercase_text+'. '
    return output_text
    tensorflow.keras.backend.clear_session()

In [16]:
def llm_processing(text):
    arr = text.split()
    output = ''
    for i in range(0,len(arr),2000):
        tt = ' '.join(arr[i:i+2000])

        inputs = [ 'Найти в тексте все термины и дай им определения на основании текста.'\
              'текст:'+tt]

        conversation = Conversation()
        conversation.add_user_message(inputs)
        prompt = conversation.get_prompt(tokenizer_saiga)
        out = generate(model_saiga, tokenizer_saiga, prompt, generation_config_saiga)
        #print(out)
        output+=out
    return output

In [20]:
import yadisk
import time

def preprocessing(file_path):
    text = vosk_analysis(file_path)
    text = ru_punct_analysis(text)   
    gloss = llm_processing(text)
    with open('out.txt', 'w') as out_f:
        out_f.write('ГЛОССАРИЙ\n'+gloss+'\nКОНСПЕКТ\n'+text)
    return 'out.txt'
    

token = 'y0_AgAAAAAzSxLyAArg1wAAAADy4fApYeNMFE5WS0aalIrIJ00O2B_X2KI'
y = yadisk.YaDisk(token=token)
while True:
    print('Слушаю')
    files = list(map(lambda x: x.name,y.listdir("app:/")))
    if 'input.mp3' in files:
        print('Начинаю обработку')

        y.download("app:/input.mp3","input.mp3")
        y.remove("app:/input.mp3", permanently=True)
        y.upload(preprocessing('input.mp3'), "app:/out.txt",overwrite=True)
        print('Отправил')
        
    else:
        time.sleep(10)

Слушаю
Начинаю обработку


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 31.74 GiB total capacity; 7.45 GiB already allocated; 2.12 MiB free; 7.97 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [22]:
torch.cuda.empty_cache()
